In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from fl_g13.editing.sparseSGDM import SparseSGDM
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import CosineAnnealingLR

import flwr
from flwr.simulation import run_simulation
from fl_g13.architectures import BaseDino
from fl_g13.fl_pytorch import get_client_app, get_server_app
from fl_g13.fl_pytorch import build_fl_dependencies

print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

build_fl_dependencies() #! Remind to always put this, it will download Dino dependencies for client

2025-05-11 09:32:24.341 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: /home/massimiliano/Projects/fl-g13


Flower 1.17.0 / PyTorch 2.6.0+cu124
'vision_transformer.py' already exists.
'utils.py' already exists.


In [3]:
# Settings
CHECKPOINT_DIR = "/home/massimiliano/Projects/fl-g13/checkpoints"

# Model hyper-parameters
head_layers=3
head_hidden_size=512
dropout_rate=0.0
unfreeze_blocks=1

# Training hyper-parameters
starting_lr = 1e-3
momentum = 0.9
weight_decay=1e-5
T_max=8
eta_min=1e-5

# Federated Training setting
batch_size = 64 # Batch size for training #! Let's stick to 64 to make training fit also on RTX 3070
local_epochs = 2 # Number of local epochs per client
number_of_rounds = 5 # Total number of federated learning rounds
fraction_fit = 1 # Fraction of clients participating in training per round
fraction_evaluate = 0.1 # Fraction of clients participating in evaluation per round
number_of_clients = 3 # Total number of clients in the simulation
min_num_clients = 2 # Minimum number of clients required for training and evaluation
partition_type = "iid" # Partitioning strategy for the dataset (e.g., "iid" or "shard")
num_shards_per_partition = 6 # Number of shards per partition (used when partition_type is "shard")
use_wandb = False # Whether to use Weights & Biases (wandb) for experiment tracking (#!TODO, double check it works)

# Device settings
device = "cuda" if torch.cuda.is_available() else "cpu"
backend_config = {
    "client_resources": {
        "num_cpus": 1, 
        "num_gpus": 0
    }
}

# When running on GPU, assign an entire GPU for each client
# Refer to Flower framework documentation for more details about Flower simulations
# and how to set up the `backend_config`
if device == "cuda":
    backend_config["client_resources"] = {"num_cpus": 1, "num_gpus": 1}

print(f"Training on {device}")

Training on cuda


In [4]:
# Model
model = BaseDino(
    head_layers=head_layers, 
    head_hidden_size=head_hidden_size, 
    dropout_rate=dropout_rate, 
    unfreeze_blocks=unfreeze_blocks
    )
model.to(device)

mask = [torch.ones_like(p, device=p.device) for p in model.parameters()] # Must be done AFTER the model is moved to CUDA
optimizer = SparseSGDM(
    model.parameters(),
    mask=mask,
    lr=starting_lr,
    momentum=momentum,
    weight_decay=weight_decay
    )
scheduler = CosineAnnealingLR(
    optimizer=optimizer, 
    T_max=T_max, 
    eta_min=eta_min
    )
criterion = CrossEntropyLoss()

client_app = get_client_app(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    device=device, 
    partition_type=partition_type, 
    batch_size=batch_size,
    num_shards_per_partition=num_shards_per_partition,
    local_epochs=local_epochs,
    model_editing=False,
)
server_app = get_server_app(
    checkpoint_dir=CHECKPOINT_DIR,
    prefix='aron', #! Introduced, you are force to pass this to avoid overwrites, if you pass an already used name it will load the most recent checkpoint
    model_class=model.__class__,
    model_config=model.get_config(), 
    optimizer=optimizer, 
    criterion=criterion, 
    scheduler=scheduler,
    device=device, 
    save_every=1,
    save_with_model_dir=True, #! Introduced: will save under {checkpoint path provided}/BaseDino/ dir if set to True
    num_rounds=number_of_rounds, 
    fraction_fit=fraction_fit,
    fraction_evaluate=fraction_evaluate,
    min_fit_clients=min_num_clients,
    min_evaluate_clients=min_num_clients,
    min_available_clients=number_of_clients,
    use_wandb=False,
    wandb_config=None,
)

Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


🔍 Loading checkpoint from /home/massimiliano/Projects/fl-g13/checkpoints/BaseDino/fl_aron_BaseDino_epoch_2.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 1, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main
Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from /home/massimiliano/Projects/fl-g13/checkpoints/BaseDino/fl_aron_BaseDino_epoch_2.pth, resuming at epoch 3


In [ ]:
run_simulation(
    client_app=client_app,
    server_app=server_app,
    num_supernodes=number_of_clients,
    backend_config=backend_config,
)

[Server] Server on device: cuda:0
[Server] CUDA available in client: True


INFO :      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


[Server Eval Round 0] Model device: cuda:0
[Server Eval Round 0] CUDA available in server eval: True


Eval progress: 100%|██████████| 313/313 [00:24<00:00, 12.97batch/s]
INFO :      [Round 0] Centralized Evaluation - Loss: 0.9655, Metrics: {'centralized_accuracy': 0.7188}
INFO :      initial parameters (loss, other metrics): 0.9654530352963426, {'centralized_accuracy': 0.7188}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=28351) 2025-05-11 09:32:53.540 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: /home/massimiliano/Projects/fl-g13


(ClientAppActor pid=28351) [Client] Client on device: cuda:0
(ClientAppActor pid=28351) [Client] CUDA available in client: True
(ClientAppActor pid=28351) No prefix/name for the model was provided, choosen prefix/name: sleepy_pidgeotto_74
(ClientAppActor pid=28351) 


...

Training progress: 100%|██████████| 209/209 [00:37<00:00,  5.60batch/s]


(ClientAppActor pid=28351) [Client] Client on device: cuda:0
(ClientAppActor pid=28351) [Client] CUDA available in client: True
(ClientAppActor pid=28351) No prefix/name for the model was provided, choosen prefix/name: itchy_nidoqueen_22
(ClientAppActor pid=28351) 


Training progress:   0%|          | 0/209 [00:00<?, ?batch/s]


(ClientAppActor pid=28351) 🚀 Epoch 1/2 (50.00%) Completed
(ClientAppActor pid=28351) 	📊 Training Loss: 0.7292
(ClientAppActor pid=28351) 	✅ Training Accuracy: 78.03%
(ClientAppActor pid=28351) 	⏳ Elapsed Time: 37.09s | ETA: 37.09s
(ClientAppActor pid=28351) 	🕒 Completed At: 09:39
(ClientAppActor pid=28351) 


Training progress:  64%|██████▎   | 133/209 [00:23<00:13,  5.70batch/s]
